A clique is a subset of a graph that each vertex is interconnected.

A maximal clique is a clique that has reached its maximum degree.

No extra vertex can be added into the clique so that each vertex is interconnected.

Bron-Kerbosch algorithm is the most efficient way to obtain maximal cliques from a graph.

In [1]:
import copy
import random as rd

In [2]:
#new functions are created for this script
#including vertex degree, adjacency matrix and remove vertex
class graph:
    def __init__(self):
        self.graph={}
        self.visited={}        

    def append(self,vertexid,edge,weight):
        if vertexid not in self.graph.keys():          
            self.graph[vertexid]={}
            self.visited[vertexid]=0
        self.graph[vertexid][edge]=weight

    def reveal(self):
        return self.graph
    
    def vertex(self):
        return list(self.graph.keys())

    def edge(self,vertexid):
        return list(self.graph[vertexid].keys())
    
    def weight(self,vertexid,edge):
        
        return (self.graph[vertexid][edge])
    
    def size(self):
        return len(self.graph)
    
    def visit(self,vertexid):
        self.visited[vertexid]=1
    
    def go(self,vertexid):
        return self.visited[vertexid]
    
    def route(self):
        return self.visited
    
    def degree(self,vertexid):
        return len(self.graph[vertexid])
    
    def mat(self):        
        self.matrix=[]
        for i in self.graph:
            self.matrix.append([0 for k in range(len(self.graph))])
            for j in self.graph[i].keys():        
                self.matrix[i-1][j-1]=1
        return self.matrix
    
    def remove(self,node):
        for i in self.graph[node].keys():
            self.graph[i].pop(node)
        self.graph.pop(node)

In [3]:
#to get maximal clique
#we need an undirected graph
#in another word, vertices with edge connections 
#are mutually connected to each other
df=graph()
df.append(1,2,6)
df.append(1,3,5)
df.append(2,1,6)
df.append(2,4,8)
df.append(2,6,3)
df.append(3,1,5)
df.append(3,4,2)
df.append(3,5,7)
df.append(4,2,8)
df.append(4,3,2)
df.append(4,5,7)
df.append(5,3,3)
df.append(5,4,7)
df.append(5,7,9)
df.append(6,2,3)
df.append(6,7,5)
df.append(7,5,9)
df.append(7,6,5)
df.append(7,8,13)
df.append(8,7,13)

![alt text](./preview/minimum%20spanning%20tree%20origin.jpg)

In [4]:
df.reveal()

{1: {2: 6, 3: 5},
 2: {1: 6, 4: 8, 6: 3},
 3: {1: 5, 4: 2, 5: 7},
 4: {2: 8, 3: 2, 5: 7},
 5: {3: 3, 4: 7, 7: 9},
 6: {2: 3, 7: 5},
 7: {5: 9, 6: 5, 8: 13},
 8: {7: 13}}

In [5]:
df.mat()

[[0, 1, 1, 0, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 1, 0, 0],
 [1, 0, 0, 1, 1, 0, 0, 0],
 [0, 1, 1, 0, 1, 0, 0, 0],
 [0, 0, 1, 1, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 1, 0]]

In [6]:
#using brute force to solve maximal clique problem
#as maximal clique problem is np complete
#back tracking algorithm has very high time complexity
#the idea is to iterate all subsets and combinations
#and find out which is a maximal clique

#clique_min defines the minimum size of maximal clique
#in practical case, we dont want to include a two edge clique
def backtrack(graph,clique_min=2):
    
    #each vertex in the graph acts as a pivot
    #we form a new tree to check all the subsets that contain pivot
    queue=graph.vertex()
    
    #output stores all the maximal cliques
    output=[]
    
    #visited keeps track of all the pivot vertices we checked
    visited=[]

    for i in queue:
        
        #check all the vertices adjacent to the pivot i
        adjacency=graph.edge(i)

        for j in adjacency:
            
            #each edge that connects two vertices can form a clique
            clique=[i,j]
            
            #check all the vertices adjacent to both pivot i and current node j
            intersection=[node for node in graph.edge(j) if node in adjacency]
            
            #to remove duplicates and reduce iterations
            #we check the reverse order as well
            if f'{i}-{j}' in visited or f'{j}-{i}' in visited:
                continue

            visited.append(f'{i}-{j}')
            
            #the current node becomes k
            #both i and j are already in the clique
            #now we have to find a maximal clique
            for k in intersection:
                
                stop=False
                
                #if node k is adjacent to all the nodes in the existing clique
                for l in clique:
                    if k not in graph.edge(l):
                        stop=True
                        break

                if stop:
                    continue
                else:
                    clique.append(k)
            
            #minimum clique size is 2 by default
            #basically all edges are included
            #as this is an undirected graph
            #we can set it to 3 to be more useful
            #use sorted to avoid duplicates in the output
            if len(clique)>=clique_min and sorted(clique) not in output:
                output.append(sorted(clique))
                
    return output

In [7]:
backtrack(df)

[[1, 2], [1, 3], [2, 4], [2, 6], [3, 4, 5], [5, 7], [6, 7], [7, 8]]

&nbsp;

### Bron Kerbosch

&nbsp;

I always admire how people write an elegant recursive algorithm. 

Bron Kerbosch is one of the most elegant algorithms. 

It takes less than 10 lines to replicate the process of my backtracking function.

The implementation comes from the pseudocode on Wikipedia.

https://en.wikipedia.org/wiki/Bron%E2%80%93Kerbosch_algorithm

If you need more detailed explanation, you can check the link below.

https://iq.opengenus.org/bron-kerbosch-algorithm/

If you have trouble understanding recursion, you can check my repository.

https://github.com/je-suis-tm/recursion-and-dynamic-programming

&nbsp;
#### Without Pivoting
&nbsp;

In [8]:
#in this function, P stands for priority queue, where pending vertices are
#R stands for result set, X stands for checked list
def bron_kerbosch(df,R=set(),P=set(),X=set()):

    #when we have nothing left in the priority queue and checked list
    #by definition, no extra vertex can be added into the clique
    #therefore, we find a maximal clique
    if not P and not X:
        yield R
        
    #while we still got vertices in priority queue
    #we pick a random adjacent vertex and add into the clique
    while P:
        
        v=P.pop()
        
        #the crucial part of the algorithm is here
        yield from bron_kerbosch(df,
                                 
                                 #we add a new adjacent vertex into the result set
                                 #trying to expand the clique to the maximal
                                 R=R.union([v]),
                                 
                                 #the priority queue is bounded by the rule of adjacency
                                 #a vertex can be added into the priority queue
                                 #if and only if it is neighbor to everyone in the current clique
                                 P=P.intersection(df.edge(v)),
                                 
                                 #the checked list is bounded by the rule of adjacency as well
                                 #hence, we can minimize the iteration we need
                                 X=X.intersection(df.edge(v)))
        
        #the vertex has been checked
        X.add(v)

In [9]:
print(list(bron_kerbosch(df,P=set(df.vertex()))))

[{1, 2}, {1, 3}, {2, 4}, {2, 6}, {3, 4, 5}, {5, 7}, {6, 7}, {8, 7}]


&nbsp;
#### With Pivoting
&nbsp;

In [10]:
def bron_kerbosch_pivot(df,R=set(),P=set(),X=set()):

    if not P and not X:
        yield R
    
    #the crucial part of pivoting is here
    #we choose a pivot vertex u from the union of pending and processed vertices
    #we delay the neighbors of pivot vertex from being added to the clique 
    #of course they will be added in the future recursive calls
    #we do that, we can make fewer iterations in recursion
    #if you count the recursive calls
    #you will find out pivot version reduce 2 recursive calls
    try:
        u=list(P.union(X)).pop()
        N=P.difference(df.edge(u))
    
    #sometimes our choice sucks
    #the neighbors of pivot u are equivalent to priority queue
    #in that case we just roll back to the function without pivoting
    except IndexError:
        N=P
    
    for v in N:
        
        yield from bron_kerbosch_pivot(df,
                                       R=R.union([v]),
                                       P=P.intersection(df.edge(v)),
                                       X=X.intersection(df.edge(v)))
        P.remove(v)
        X.add(v)

In [11]:
print(list(bron_kerbosch_pivot(df,P=set(df.vertex()))))

[{1, 2}, {1, 3}, {2, 4}, {2, 6}, {3, 4, 5}, {5, 7}, {6, 7}, {8, 7}]


&nbsp;
#### With Vertex Ordering
&nbsp;

In [12]:
#please check the details of degeneracy ordering from k core
# https://github.com/je-suis-tm/graph-theory/blob/master/k%20core.ipynb
def get_degree_list(df):

    D={}
    
    for i in df.vertex():
        try:
            D[df.degree(i)].append(i)

        except KeyError:
            D[df.degree(i)]=[i]
    
    D=dict(sorted(D.items()))
    
    return D

#we only need degeneracy ordering L
#the whole function has been simplified
def matula_beck(df):
    
    subset=copy.deepcopy(df)
    
    L=[]

    D=get_degree_list(subset)

    while D:
        
        i=list(D.keys())[0]
        
        v=D[i].pop(0)

        L.append(v)
        subset.remove(v)
        
        D=get_degree_list(subset)   
    
    return L

In [13]:
#the easiest way to understand is to think of k core
#a maximal clique suffices to the condition of a k core network
#because a maximal clique is fully connected
#each vertex inside is guaranteed to have degree of k
#where k+1 is the order of the maximal clique
#but the reverse does not hold
#each vertex is ranked by its guaranteed degree in degeneracy ordering
#when we choose vertex from degeneracy ordering
#we output the maximal cliques in ascending order
#the later vertices extracted from degeneracy ordering
#are more likely to form a maximal clique of a larger degree altogether
#in this way, we can potentially reduce the iteration
def bron_kerbosch_order(df,R=set(),P=set(),X=set()):
    
    deg_order=matula_beck(df)
    
    if not P and not X:
        yield R
    
    for v in deg_order:
        yield from bron_kerbosch_pivot(df,
                                       R=R.union([v]),
                                       P=P.intersection(df.edge(v)),
                                       X=X.intersection(df.edge(v)))
        P.remove(v)
        X.add(v)

In [14]:
#if you print degeneracy ordering
#you will notice 3,4,5 is at the bottom of it
#and 3,4,5 forms a perfect maximal clique
print(list(bron_kerbosch_order(df,P=set(df.vertex()))))

[{8, 7}, {1, 2}, {1, 3}, {2, 4}, {2, 6}, {6, 7}, {5, 7}, {3, 4, 5}]


&nbsp;
#### Time Complexity Comparison
&nbsp;

In [15]:
#this is a maximum clique with degree 100
df=graph()

for i in range(1,101):
    for j in range(1,101):
        if i!=j:
            df.append(i,j,0)

In [16]:
#outside of the maximum clique
#we add more one more layer
for i in range(1,101):
    df.append(i,i+100,0)
    df.append(i+100,i,0)

In [17]:
#outside the first layer
#we randomly add a second layer
#which should construct some new maximal cliques with degree 2
for i in range(1,101):
    if rd.choice([0,1])==1:
        df.append(i,i+200,0)
        df.append(i+200,i,0)
        df.append(i+200,i+100,0)
        df.append(i+100,i+200,0)
    else:
        df.append(i+100,i+200,0)
        df.append(i+200,i+100,0)

In [18]:
#here comes the third layer of the random graph
#it provides the chance to form some new maximal cliques with degree 3
for i in range(1,101):
    if rd.choice([0,1])==1:
        df.append(i,i+300,0)
        df.append(i+300,i,0)
        df.append(i+300,i+100,0)
        df.append(i+100,i+300,0)
        df.append(i+200,i+300,0)
        df.append(i+300,i+200,0)
    else:
        df.append(i+100,i+300,0)
        df.append(i+300,i+100,0)

In [19]:
%timeit bron_kerbosch(df,P=set(df.vertex()))

12.2 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [20]:
%timeit bron_kerbosch_pivot(df,P=set(df.vertex()))

12.2 µs ± 68.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [21]:
%timeit bron_kerbosch_order(df,P=set(df.vertex()))

12.1 µs ± 77.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


&nbsp;
### Further Reading
&nbsp;
1. Bron, C. and J. Kerbosch (1971), "Finding All Cliques of an Undirected Graph"

https://www.ics.uci.edu/~johnsong/papers/Bron%20and%20Kerbosch%20-%20Finding%20All%20Cliques%20of%20an%20Undirected%20Graph.pdf

2. Eppstein, D., M. Löffler and D. Strash (2010), "Listing All Maximal Cliques in Sparse Graphs in Near-optimal Time"

https://arxiv.org/abs/1006.5440
    
3. Tomitaa, E., A. Tanakaa and H. Takahashia (2006), "The worst-case time complexity for generating all maximal cliques and computational experiments"

https://www.sciencedirect.com/science/article/pii/S0304397506003586